In [18]:
import torch
import pandas as pd
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader

In [19]:
file_name = 'fox-4-abortion'
df = pd.read_csv(f"/home/siu856533724/code/source-code/Social-Networks/Trend-Prediction/DataSet/reply-tree/csv/{file_name}.csv")

In [20]:
df.head()

,Unnamed: 0,comment,time,username,id,timestamp,time_diff
0,64,You will have a whole a lot of accountability ...,"7 Jan, 2024 07:13 AM",punion,65,2024-01-07 07:13:00,0 days 00:00:00
1,243,Abortions = the new legal way to murder the in...,"7 Jan, 2024 07:17 AM",victoria080,244,2024-01-07 07:17:00,0 days 00:04:00
2,301,maybe you should have had your many partners w...,"7 Jan, 2024 07:18 AM",ZombiesEatBrainsLibsRsafe,302,2024-01-07 07:18:00,0 days 00:05:00
3,26,The author needs to learn to use an aspirin fo...,"7 Jan, 2024 07:19 AM",buzzard10264,27,2024-01-07 07:19:00,0 days 00:06:00
4,134,Four Abortions? You should be locked up.,"7 Jan, 2024 07:19 AM",45twice,135,2024-01-07 07:19:00,0 days 00:06:00


In [21]:
df = df.sort_values('id')

In [22]:
df.head()

,Unnamed: 0,comment,time,username,id,timestamp,time_diff
34,0,"""The options are vast for women facing unplann...","7 Jan, 2024 07:44 AM",scruffypup,1,2024-01-07 07:44:00,0 days 00:31:00
741,1,It's too bad Republicans always try to fight a...,"7 Jan, 2024 06:45 PM",🇺🇸USA🇺🇸🇮🇱Israel🇮🇱🇺🇦Ukraine🇺🇦,2,2024-01-07 18:45:00,0 days 11:32:00
749,2,Are they fighting all birth control or just ab...,"7 Jan, 2024 07:09 PM",scruffypup,3,2024-01-07 19:09:00,0 days 11:56:00
738,3,Exactly.,"7 Jan, 2024 06:26 PM",teachtolove123,4,2024-01-07 18:26:00,0 days 11:13:00
125,4,This is such a heartbreaking and devastating p...,"7 Jan, 2024 08:20 AM",BFDfromSC,5,2024-01-07 08:20:00,0 days 01:07:00


In [23]:
trends = ["approval","toxic","obscene", 'insult', "threat", "hate", "offensive", "neither"]

In [24]:
from utils.common_functions import get_emotions
emotions = get_emotions().values()
n_labels = len(emotions)
def get_labels():
    labels = list(emotions)
    return labels

In [25]:
def classifier(data_loader, model, device):
    model.eval()
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            mask = d["mask"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)

            outputs = model(ids, attention_mask=mask)

            fin_outputs.extend(torch.nn.functional.softmax(outputs.logits))

    return fin_outputs

In [26]:
from utils.dataset import Dataset

def build_dataset(tokenizer_max_len, tokenizer, is_labels):
    dataset = Dataset(df.comment.tolist(), df[trends].values.tolist() if is_labels else [], tokenizer, tokenizer_max_len)
    return dataset
    # return train_dataset, valid_dataset

def build_dataloader(dataset, batch_size):
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=1)
    return data_loader
    # return train_data_loader, valid_data_loader

In [27]:
from utils.bert_classifier import get_model_tokenizer, Tokenize, Classification
# Test sentiment prediction
# test_text = "I love you"
model_emotion, tokenizer_emotion = get_model_tokenizer(pred_type='emotion_pred', mode_pt_file='best_emotion_bert-base-uncased-20_12_2023-23_14_48.pt', 
                                       model_ckpt='bert-base-uncased', labels=28, is_freez=1, gpu='cuda')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [28]:
from utils.test import test_fn

dataset = build_dataset(tokenizer_max_len=30, tokenizer=tokenizer_emotion, is_labels=False)
data_loader = build_dataloader(dataset, batch_size=16)
preds = classifier(data_loader=data_loader, model=model_emotion, device='cuda')

preds = torch.stack(preds)
preds = preds.cpu().detach().numpy()

  0%|          | 0/48 [00:00<?, ?it/s]

In [29]:
map_list = []
labels = get_labels()
for i, t in enumerate(range(len(preds))):
    data_map = {}
    # data_map['id'] = id
    data_map['id'] = (i+1)
    for j, label in enumerate(labels):
        data_map[label] = preds[i][j]
    map_list.append(data_map)

df_fox_emotion = pd.DataFrame(data=map_list)

In [30]:
df_fox_emotion.to_csv(f'/home/siu856533724/code/source-code/Social-Networks/Trend-Prediction/DataSet/reply-tree/csv/{file_name}-emotions.csv')

## Trend Prediction

In [31]:
model_trend, tokenizer_trend = get_model_tokenizer(pred_type='trend_pred', mode_pt_file='best_trend_bert-base-uncased-18_12_2023-12_49_37.pt', 
                                       model_ckpt='bert-base-uncased', labels=8, is_freez=1, gpu='cuda')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [32]:
dataset = build_dataset(tokenizer_max_len=30, tokenizer=tokenizer_trend, is_labels=False)
data_loader = build_dataloader(dataset, batch_size=16)
preds = classifier(data_loader=data_loader, model=model_trend, device='cuda')
preds = torch.stack(preds)
preds = preds.cpu().detach().numpy()

  0%|          | 0/48 [00:00<?, ?it/s]

In [33]:
map_list = []
labels = ["approval","toxic","obscene", 'insult', "threat", "hate", "offensive", "neither"]
for i, t in enumerate(range(len(preds))):
    data_map = {}
    # data_map['id'] = id
    data_map['id'] = (i+1)
    for j, label in enumerate(labels):
        data_map[label] = preds[i][j]
    map_list.append(data_map)

df_fox_trend = pd.DataFrame(data=map_list)

In [34]:
df_fox_trend.to_csv(f'/home/siu856533724/code/source-code/Social-Networks/Trend-Prediction/DataSet/reply-tree/csv/{file_name}-trends.csv')